In [1]:
#import libraries
import pandas as pd
import numpy as np
import nltk 
from nltk.classify.textcat import TextCat 
from matplotlib.pyplot import plot 
from datascience import *

In [1]:
#fuzzywuzzy
!pip3 install fuzzywuzzy[speedup]
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#(py)torch
!pip install torch

#langid (language identification)
!pip install langid
import langid

In [4]:
#DO NOT RUN 
#This code creates a traditional table as seen in the datascience * package but takes up too much memory. 
#I suggest running the code block after this one that generates chunks of 10 for each row of data.

chatbots_data = Table.read_table("chatbots_data.csv")

#Convert Pandas dataframe to Table (where you can use table.select(...)). Not suggested -- takes up lots of memory
chatbots_data = Table.from_df(chatbots_data_chunks, keep_index=False)

chatbots_data

In [2]:
#dividing the data into chunks of 10, because of memory issues (kernel keeps dying)
#for example, this is what one chunk looks like
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    print(chunk.shape)
    print("="*66)
    print(chunk.head(2))
    print("="*66)
    break
    
#appending chunks to chatbot_list, an empty array
chatbot_list = []
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    chatbot_list.append(chunk)

#Combine all the chunks into one dataframe
chatbots_data_chunks = pd.concat(chatbot_list, axis=0)
chatbots_data_chunks = pd.DataFrame(data = chatbots_data_chunks)

In [5]:
#cleans the "Message" column
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"").replace('_',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

chatbots_data_chunks['Message']=chatbots_data_chunks['Message'].map(lambda s:preprocess(s)) 

In [3]:
#view chunks
chatbots_data_chunks

In [5]:
#View cleaned data, all 80,000+ rows

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
chatbots_data_chunks

In [5]:
#collocation finder....might scrap later
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(chatbots_data_chunks["Message"].values.reshape(-1, ))

bigrams that appear 3+ times
finder.apply_freq_filter(2)

 return the 10 n-grams with the highest PMI (pointwise mutual information)
finder.nbest(bigram_measures.pmi, 10)

In [ ]:
#Backend portion...find the most common phrases 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(chatbots_data_chunks["Message"])  
features = (vectorizer.get_feature_names()) 
#print("\n\nFeatures : \n", features) <-- if you want to see all the phrases
#print("\n\nX1 : \n", X1.toarray())   <-- but I don't recommend running
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(chatbots_data_chunks["Message"]) 
scores = (X2.toarray()) 
#print("\n\nScores : \n", scores) <--- I don't recommend running

In [9]:
# Getting top 20 ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
# Alternatively, put "True" here to see the most uncommon trigrams

common_phrases =  print("\n\nWords head : \n", words.head(20)) 
common_phrases

In [7]:
#backend part that allows fuzzy to work
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

choices = chatbots_data_chunks['Message'].unique()

In [10]:
# much services cost
# it appears that users are curious about the services the bots provide, as well as how much those services might cost
# however, fuzzywuzzy is also pulling "STIs" for "services" which could be an issue
process.extract("much services cost", choices, limit = 10, scorer = fuzz.token_sort_ratio)

In [11]:
#change mind sex
process.extract("change mind sex", choices, limit = 10, scorer = fuzz.token_sort_ratio)

In [12]:
#know ready sex
process.extract("know ready sex", choices, limit = 10, scorer = fuzz.token_sort_ratio)

In [14]:
nltk.download('crubadan')
#N-Gram based categorization, identifying languages...which is very off...

tc = TextCat()
chatbots_data_chunks["Message"][0:5].apply(tc.guess_language)

In [15]:
import numpy as np
#from scipy.sparse import csr_matrix
!pip install sparse_dot_topn
import sparse_dot_topn.sparse_dot_topn as ct

In [8]:
# backend portion for string matching
from sklearn.feature_extraction.text import TfidfVectorizer

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

message = chatbots_data_chunks["Message"]
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(message)

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
    idx_dtype = np.int32
    nnz_max = M*ntop
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1

def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

In [16]:
#string-matching output
matches_df = get_matches_df(matches, message, top=100000)
matches_df = matches_df[matches_df['similarity'] < 0.99999] # Remove all exact matches
matches_df.sample(50)

In [ ]:
# Plan ahead:
# Cluster data with Levenshtein distance: i.e., "change myvmind sex" is matched to "change mind sex"
# Find a better language identification algorithm, so bots don't mistake different languages as mispelled words
# Q: Which users are asking the most about which subject?